Below serve as example on how this script should look like and not represent the full script

In [1]:
# import related libraries

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-mongo-streaming')     
         .config("add spark, kafka,mongodb package here. ensure version is compatible")
         # Mongo config including the username and password from compose file
         .config("enter mongodb uri here with related credentials")
         .getOrCreate())

In [2]:
# Read the message from the kafka stream
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka related port") \
  .option("subscribe", "created kafka topic") \
  .load()

# convert the binary values to string
df1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [3]:
#Create a temporary view for SparkSQL
df1.createOrReplaceTempView("query")

In [4]:
# Write out the message to the console of the environment
res = spark.sql("your SQL statement")
res.writeStream.format("console") \
            .outputMode("append") \
            .start()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, struct
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

def foreach_batch_function(df, epoch_id):
    # Define schemas for nested JSON fields
    card_holder_schema = StructType([
        StructField("first", StringType(), True),
        StructField("last", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("dob", StringType(), True)
    ])
    #Transform the values of all rows in column value and create a dataframe out of it (will also only have one row)
    df2=df.withColumn("value",from_json(df.value,"do your transformation function here"))
    # Transform the dataframe so that it will include individual columns based on the BaseModel structure
    df3 = df2.select(
        col("value.trans_num").alias("trans_num"),
        col("value.trans_date_trans_time").alias("trans_date_trans_time"),
        col("value.cc_num").alias("cc_num"),
        col("value.merchant").alias("merchant"),
        col("value.category").alias("category"),
        col("value.amt").alias("amt")
    )
    
    # Transform the DataFrame
    transformed_df = df3.select(
        col("trans_num"),
        col("trans_date_trans_time"),
        col("cc_num"),
        col("merchant"),
        col("category"),
        col("amt").cast(DoubleType())
    )
    
    transformed_df.write.format("related mongodb spark format source").mode("related mode to save json").save()

In [ ]:
# use above function to run spark microbatch
# Start the MongoDB stream and wait for termination